In [ ]:
# default_exp simulation

# Simulation

> Tools for  simulating cellular vertex dynamics. Iterative algorithm implementations, cell monolayer generators (vertices). (anything else to add?!)

- numerical integration method(s)
    - Euler's method
    - any other approximation?
- implement functions for checking:
    - vertex configuration (for high tension cases there might be overlapping, or twisted cells which are considered illigal by the model)
    - update steps that implement required vertex check, and any other checks needed before accepting vertex position update
- implement Voronoi cell generator for different cell configuration cases (random, evenly spaced, concave cell, ...)

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from vertex_simulation.primitives import *
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import torch, numpy as np, matplotlib.pyplot as plt
from scipy.spatial import Voronoi,voronoi_plot_2d

## Euler's method

In [ ]:
# define function for numerical integration

In [ ]:
#hide
# run this as a last cell in your notebook to export this module
from nbdev.export import *
notebook2script()

Converted 00_primitives.ipynb.
Converted 01_simulation.ipynb.
Converted index.ipynb.
Converted vertex_dynamics_models.ipynb.
